<a href="https://colab.research.google.com/github/BlackWyvernX/GPCR-Ligand-Dock/blob/main/Docking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q py3Dmol==2.0.3
!pip install -q pybel==0.15.5
!pip install -q rdkit-pypi==2022.9.5
!pip install -q meeko

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge()

In [ ]:
%%bash
mkdir vina
cd vina
wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64
chmod +x /content/vina/vina_1.2.5_linux_x86_64
wget -q https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_split_1.2.5_linux_x86_64
chmod +x /content/vina/vina_split_1.2.5_linux_x86_64

mkdir: cannot create directory ‘vina’: File exists


In [ ]:
%alias vina /content/vina/vina_1.2.5_linux_x86_64
%alias vina_split /content/vina/vina_1.2.5_linux_x86_64

In [ ]:
!mamba install -c conda-forge -c bioconda mgltools=1.5.7 biopython=1.80 \
openbabel=3.1.1 plip=2.2.2 zlib=1.2.13 xlsxwriter=3.0.3 fpocket

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages/')

import ast
import math
import plip
import timeit
import shutil
import py3Dmol
import contextlib
import xlsxwriter
import urllib.request
import subprocess
import warnings

import requests as rq
import pandas as pd
import os
import json

import numpy as np
import pandas as pd


from google.colab import drive, files
from tqdm.notebook import tqdm
from openbabel import pybel
from Bio.PDB import PDBIO, PDBParser
from Bio.PDB import *
from rdkit import Chem
from rdkit.Chem import rdFMCS, AllChem, PandasTools
from plip.exchange.report import BindingSiteReport
from plip.structure.preparation import PDBComplex


In [ ]:
job = '7y89'
DIR = os.getcwd()
WRK_DIR = os.path.join(DIR, job)
EXP_FLD = os.path.join(WRK_DIR, "EXPERIMENTAL")
DCK_FLD = os.path.join(WRK_DIR, "DOCKING")
INT_FLD = os.path.join(WRK_DIR, "INTERACTION")
temp = os.path.join(DIR, 'temp')
liga = os.path.join(DIR, 'ligand')

folders = [WRK_DIR, EXP_FLD, DCK_FLD, INT_FLD, temp, liga]

for f in folders:
    if os.path.exists(f):
        print(f"+ {os.path.basename(f)} folder already exists")
    if not os.path.exists(f):
        os.mkdir(f)
        print(f"+ {os.path.basename(f)} folder created")


+ 7y89 folder created
+ EXPERIMENTAL folder created
+ DOCKING folder created
+ INTERACTION folder created
+ temp folder created
+ ligand folder created


In [ ]:
pdbid = ['7y89']
pdbl = PDBList()
for s in pdbid:
  pdbl.retrieve_pdb_file(s, pdir='/content/7y89', file_format ="pdb", overwrite=True)
  os.rename('/content/7y89/pdb7y89.ent', "/content/7y89/7y89.pdb")

In [ ]:
!prepare_receptor4.py -r /content/7y89/7y89.pdb -o /content/7y89/7y89_receptor.pdbqt -A hydrogens -U nphs_lps -v

set verbose to  True
read  /content/7y89/7y89.pdb
setting up RPO with mode= automatic and outputfilename=  /content/7y89/7y89_receptor.pdbqt
charges_to_add= gasteiger
delete_single_nonstd_residues= None
adding gasteiger charges to peptide


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GPCR Project/GPR17Ligand.csv')

lig = df['Molecule'].values.tolist()
dir = '/content/ligand'

In [ ]:
def fetch_sdfb(ligand_names, output):

    for ligand_name in ligand_names:
        url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{ligand_name}/cids/JSON"
        response = rq.get(url)

        if response.status_code == 200:
            temp_filename = f"{ligand_name}_temp.json"
            with open(temp_filename, "wb") as temp_file:
                temp_file.write(response.content)

            try:
                with open(temp_filename, "r") as json_file:
                    data = json.load(json_file)


                if "IdentifierList" in data:
                    cid = data["IdentifierList"]["CID"][0]

                    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/SDF"
                    response = rq.get(url)

                    sdf_filename = f"{ligand_name}.sdf"
                    with open(os.path.join(output, sdf_filename), "wb") as sdf_file:
                        sdf_file.write(response.content)
                        print(f"Downloaded SDF for {ligand_name}.")
                else:
                    print(f"No CID found for {ligand_name}. Skipping.")

            except Exception as e:
                print(f"Error processing {ligand_name}: {e}")

            os.remove(temp_filename)

        else:
            print(f"Error: {response.status_code} - {response.content}")


In [ ]:
fetch_sdfb(lig, dir)

Downloaded SDF for SCHEMBL9186131.
Downloaded SDF for CHEMBL3968577.
Downloaded SDF for SCHEMBL9186177.
Downloaded SDF for CHEMBL53914.
Downloaded SDF for ASN 02563583.
Downloaded SDF for Mdl-29951.
Downloaded SDF for CHEMBL3264006.
Downloaded SDF for CHEMBL3897541.
Downloaded SDF for CHEMBL3132880.
Downloaded SDF for Pranlukast.
Downloaded SDF for CHEMBL3132881.
Downloaded SDF for CHEMBL3132886.
Downloaded SDF for CHEMBL3132882.
Downloaded SDF for CHEMBL50976.
Downloaded SDF for CHEMBL3915176.
Downloaded SDF for CHEMBL3132883.
Downloaded SDF for Gavestinel.
Downloaded SDF for CHEMBL3132887.
Error: 503 - b'{\n  "Fault": {\n    "Code": "PUGREST.ServerBusy",\n    "Message": "Too many requests or server too busy"\n  }\n}\n'
Downloaded SDF for CHEMBL3921330.
Downloaded SDF for 3-(2-carboxyethyl)-6-chloro-1H-indole-2-carboxylic Acid.


In [ ]:
ligands_directory = "/content/ligand"
dck_fold = "/content/7y89/DOCKING"
root = '/content/'
tempr = '/content/temp'

In [ ]:
for ligand_file in os.listdir(ligands_directory):
    if ligand_file.endswith(".sdf"):
        ligand_name = os.path.splitext(ligand_file)[0]
        ligand_sdf = os.path.join(ligands_directory, f"'{ligand_file}'")
        ligand_mol2 = os.path.join(root, f"'{ligand_name}.mol2'")

        try:
            cmd_obabel = f'obabel {ligand_sdf} -O {ligand_mol2} --gen3d --best --canonical --minimize --ff GAFF --steps 10000 --sd'

            os.system(cmd_obabel)
            print(f'{ligand_name}' " converted sucessfully")
        except Exception as e:
            print(f"Error while converting {ligand_file}: {str(e)}")
    else:
        pass


CHEMBL3132882 converted sucessfully
CHEMBL3897541 converted sucessfully
3-(2-carboxyethyl)-6-chloro-1H-indole-2-carboxylic Acid converted sucessfully
CHEMBL3132881 converted sucessfully
SCHEMBL9186131 converted sucessfully
Pranlukast converted sucessfully
CHEMBL3968577 converted sucessfully
CHEMBL3132886 converted sucessfully
CHEMBL3264006 converted sucessfully
Mdl-29951 converted sucessfully
CHEMBL53914 converted sucessfully
SCHEMBL9186177 converted sucessfully
Gavestinel converted sucessfully
CHEMBL3921330 converted sucessfully
CHEMBL3132880 converted sucessfully
CHEMBL3132887 converted sucessfully
ASN 02563583 converted sucessfully
CHEMBL3132883 converted sucessfully
leukotriene D4 converted sucessfully
CHEMBL3915176 converted sucessfully
CHEMBL50976 converted sucessfully


In [ ]:
for ligand_file in os.listdir(root):
    if ligand_file.endswith(".mol2"):
        ligand_name = os.path.splitext(ligand_file)[0]
        ligand_mol = os.path.join(root, f"'{ligand_file}'")
        ligand_pdbqt = os.path.join(dck_fold, f"'{ligand_name}.pdbqt'")

        cmd_prepare_ligand = f'prepare_ligand4.py -l {ligand_mol} -o {ligand_pdbqt} -U nphs_lps -v'

        os.system(cmd_prepare_ligand)
        print(f'{ligand_name}' " converted sucessfully")
    else:
        pass

3-(2-carboxyethyl)-6-chloro-1H-indole-2-carboxylic Acid converted sucessfully
CHEMBL53914 converted sucessfully
CHEMBL3132880 converted sucessfully
CHEMBL3264006 converted sucessfully
Gavestinel converted sucessfully
CHEMBL3132886 converted sucessfully
CHEMBL3132882 converted sucessfully
CHEMBL3921330 converted sucessfully
CHEMBL3132883 converted sucessfully
CHEMBL3897541 converted sucessfully
Pranlukast converted sucessfully
SCHEMBL9186131 converted sucessfully
leukotriene D4 converted sucessfully
Mdl-29951 converted sucessfully
SCHEMBL9186177 converted sucessfully
CHEMBL3968577 converted sucessfully
CHEMBL3915176 converted sucessfully
CHEMBL3132887 converted sucessfully
CHEMBL50976 converted sucessfully
CHEMBL3132881 converted sucessfully
ASN 02563583 converted sucessfully


In [ ]:
for ligand_file in os.listdir(root):
    if ligand_file.endswith(".mol2"):
        ligand_name = os.path.splitext(ligand_file)[0]
        try:
            os.remove(f'{root}/{ligand_file}')
        except:
            print('No such File as' f'{ligand_file}.mol2')
    else:
      pass

In [ ]:
!fpocket

In [ ]:
!fpocket -f /content/7y89/7y89.pdb -c A

***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 


In [ ]:
def pocket_score_df(file_path):
    parameters = {
        "Score": [],
        "Druggability Score": [],
        "Number of Alpha Spheres": [],
        "Total SASA": [],
        "Polar SASA": [],
        "Apolar SASA": [],
        "Volume": [],
        "Mean local hydrophobic density": [],
        "Mean alpha sphere radius": [],
        "Mean alp. sph. solvent access": [],
        "Apolar alpha sphere proportion": [],
        "Hydrophobicity score": [],
        "Volume score": [],
        "Polarity score": [],
        "Charge score": [],
        "Proportion of polar atoms": [],
        "Alpha sphere density": [],
        "Cent. of mass - Alpha Sphere max dist": [],
        "Flexibility": []
    }

    with open(file_path, 'r') as file:
        current_parameter = None

        for line in file:
            line = line.strip()
            if line.startswith("Pocket"):
                current_parameter = None

            elif ":" in line:
                key, value = [item.strip() for item in line.split(':')]
                if key in parameters:
                    current_parameter = key
                    parameters[key].append(value)
            elif current_parameter is not None:
                parameters[current_parameter][-1] += " " + line

    pocket_score = pd.DataFrame(parameters)

    Pocket_Number = ['pocket' + str(i) for i in range(1, len(pocket_score) + 1)]

    pocket_score.insert(0, "Pocket Number", Pocket_Number)

    return pocket_score

In [ ]:
pocketscorefile = '/content/7y89/7y89_out/7y89_info.txt'
pocket_score = pocket_score_df(pocketscorefile)

In [ ]:
pocket_score

,Pocket Number,Score,Druggability Score,Number of Alpha Spheres,Total SASA,Polar SASA,Apolar SASA,Volume,Mean local hydrophobic density,Mean alpha sphere radius,Mean alp. sph. solvent access,Apolar alpha sphere proportion,Hydrophobicity score,Volume score,Polarity score,Charge score,Proportion of polar atoms,Alpha sphere density,Cent. of mass - Alpha Sphere max dist,Flexibility
0,pocket1,0.128,0.000,27,39.426,23.727,15.699,155.687,0.000,3.681,0.442,0.000,20.600,3.900,7,0,55.000,2.311,4.676,0.076
1,pocket2,0.072,0.046,34,96.325,32.321,64.004,336.743,15.889,3.817,0.592,0.529,-9.000,3.700,5,1,40.000,3.846,9.370,0.258
2,pocket3,0.071,0.033,101,349.431,194.854,154.576,1297.011,15.786,4.042,0.565,0.277,5.542,4.417,20,2,47.368,8.844,23.112,0.194
3,pocket4,0.062,0.002,25,82.467,28.123,54.343,342.185,4.000,3.905,0.619,0.200,-11.667,4.444,8,4,45.455,3.556,8.283,0.355
4,pocket5,0.057,0.003,37,94.304,37.545,56.759,299.519,3.667,3.723,0.414,0.162,30.917,4.250,7,-1,48.000,3.363,8.359,0.102
5,pocket6,0.048,0.000,27,99.472,69.282,30.191,288.267,1.000,3.918,0.563,0.074,-21.167,5.833,6,2,54.545,4.007,7.555,0.275
6,pocket7,0.039,0.001,24,71.734,19.331,52.403,162.903,3.000,3.569,0.381,0.167,12.000,3.333,6,1,42.105,2.927,6.538,0.560
7,pocket8,0.033,0.004,34,76.432,26.919,49.513,254.355,8.000,3.778,0.438,0.265,28.750,4.750,6,0,38.095,2.594,6.336,0.112
8,pocket9,0.027,0.000,16,81.863,40.804,41.059,198.595,2.000,3.732,0.552,0.188,-5.429,3.000,4,-1,43.750,3.610,7.134,0.520
9,pocket10,0.022,0.011,17,114.005,36.717,77.288,325.208,6.889,3.935,0.696,0.529,3.143,3.143,4,1,40.909,4.907,12.008,0.736


In [ ]:
def get_chain_id(pdb_file):
    structure = PDBParser().get_structure('structure', pdb_file)
    for model in structure:
        for chain in model:
            for atom in chain:
                return chain.id

warnings.filterwarnings("ignore")

pocket_dir = '/content/7y89/7y89_out/pockets'
selected_pockets = []
receptor_chain_id = 'A'

for pocket_file in os.listdir(pocket_dir):
    if pocket_file.endswith('.pdb'):
        pocket_path = os.path.join(pocket_dir, pocket_file)

        pocket_chain_id = get_chain_id(pocket_path)
        if pocket_chain_id == receptor_chain_id:
          pocket_number = os.path.splitext(pocket_file)[0]
          selected_pockets.append(pocket_number.partition('_')[0])
          print(selected_pockets)

Pocket_Score_ChainA = pocket_score[pocket_score['Pocket Number'].str.split().str[0].isin(selected_pockets)]

Pocket_Score_ChainA

,Pocket Number,Score,Druggability Score,Number of Alpha Spheres,Total SASA,Polar SASA,Apolar SASA,Volume,Mean local hydrophobic density,Mean alpha sphere radius,Mean alp. sph. solvent access,Apolar alpha sphere proportion,Hydrophobicity score,Volume score,Polarity score,Charge score,Proportion of polar atoms,Alpha sphere density,Cent. of mass - Alpha Sphere max dist,Flexibility


In [ ]:
pocket_score['Pocket Number'].str.split().str[0]


In [ ]:
view=py3Dmol.view()
view.setBackgroundColor('white')

view.addModel(open('/content/7y89/7y89.pdb', 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'spectrum'}})
view.zoomTo()

def get_chain_id(pdb_file):
    structure = PDBParser().get_structure('structure', pdb_file)
    for model in structure:
        for chain in model:
            for atom in chain:
                return chain.id

warnings.filterwarnings("ignore")

pocket_dir = '/content/7y89/7y89_out/pockets'
receptor_chain_id = 'A'

for pocket_file in os.listdir(pocket_dir):
    if pocket_file.endswith('.pdb'):
        pocket_path = os.path.join(pocket_dir, pocket_file)

        # Check the chainID of the pocket
        pocket_chain_id = get_chain_id(pocket_path)
        if pocket_chain_id == receptor_chain_id:
          pocket_chainid_path = os.path.join(pocket_dir, pocket_file)
          print(pocket_chainid_path)
          view.addModel(open(pocket_chainid_path, 'r').read(),'pdb')
          view.setStyle({'model':-1},{'stick':{'colorscheme':'greenCarbon'}})

          view.show()

In [ ]:
conf = "config_file"
conf_file = os.path.join(DCK_FLD, conf)


#@title Generating an AutoDock Vina file
#receptor = "/content/7y89/7y89_receptor.pdbqt" # @param {type:"string"}
center_x = "141.074" # @param {type:"string"}
center_y = "141.074" # @param {type:"string"}
center_z = "141.074" # @param {type:"string"}
size_x = "34" # @param {type:"string"}
size_y = "34" # @param {type:"string"}
size_z = "34" # @param {type:"string"}
with open(conf_file,"w") as f:
  f.write("#CONFIGURATION FILE (options not used are commented) \n")
  f.write("\n")
  f.write("#INPUT OPTIONS \n")
  f.write("#flex = [flexible residues in receptor in pdbqt format] \n")
  f.write("#SEARCH SPACE CONFIGURATIONS \n")
  f.write("#Center of the box (values bxi, byi and bzi) \n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX CENTER COORDINATES
  f.write("center_x = " + center_x + "\n")
  f.write("center_y = " + center_y + "\n")
  f.write("center_z = " + center_z + "\n")
#CHANGE THE FOLLOWING DATA WITH YOUR BOX DIMENSIONS
  f.write("#Size of the box (values bxf, byf and bzf) \n")
  f.write("size_x = " + size_x + "\n")
  f.write("size_y = " + size_y + "\n")
  f.write("size_z = " + size_z + "\n")
  f.write("#OUTPUT OPTIONS \n")
  f.write("#out = \n")
  f.write("#log = \n")
  f.write("\n")
  f.write("#OTHER OPTIONS \n")
  f.write("#cpu =  \n")
  f.write("#exhaustiveness = \n")
  f.write("#num_modes = \n")
  f.write("#energy_range = \n")
  f.write("#seed = ")


In [ ]:
/content/7y89/7y89_out/7y89_pockets.pqr

SyntaxError: invalid decimal literal (<ipython-input-17-18236541ad2b>, line 1)

In [ ]:
ligand_directory = "/content/7y89/DOCKING"
receptor = "/content/7y89/7y89_receptor.pdbqt"

for ligand_file in os.listdir(ligand_directory):
    if ligand_file.endswith(".pdbqt"):
        ligand_name = os.path.splitext(ligand_file)[0]
        ligand_quotes = f"'{ligand_file}'"
        ligand_pdbqt = os.path.join(dck_fold, f"'{ligand_name}.pdbqt'")
        os.chdir('/content/vina')


        vina_command = f"%vina --config {'/content/7y89/DOCKING/config_file'} --receptor {receptor} --ligand {ligand_pdbqt} --out {ligand_quotes} | tee {ligand_quotes}_output.log"

        os.system(vina_command)

In [ ]:
os.chdir('/content/vina')

%vina --config /content/7y89/DOCKING/config_file --receptor /content/7y89/7y89_receptor.pdbqt --ligand /content/7y89/DOCKING/'3-(2-carboxyethyl)-6-chloro-1H-indole-2-carboxylic Acid.pdbqt' --out /content/7y89/EXPERIMENTAL/'docking_3-(2-carboxyethyl)-6-chloro-1H-indole-2-carboxylic Acid.pdbqt' | tee /content/7y89/DOCKING/'3-(2-carboxyethyl)-6-chloro-1H-indole-2-carboxylic Acid.pdbqt'_output.log

os.chdir("/content/")

In [ ]:
c_file = '/content/7y89/DOCKING/config_file'
receptor_file = '/content/7y89/7y89_receptor.pdbqt'


for ligand_file in os.listdir(dck_fold):
    if ligand_file.endswith(".pdbqt"):
        ligand_path = os.path.join(dck_fold, f"'{ligand_file}'")
        output_path = os.path.join(EXP_FLD, f"'docking_{ligand_file}'")
        tee = os.path.join(dck_fold, f"'{ligand_file}'_output.log")

        vina = f'%vina --config {c_file} --receptor {receptor_file} --ligand {ligand_path} --out {output_path} | tee {tee}'
        print(vina)

       try:
            os.chdir('/content/vina')
            subprocess.call(vina, shell=True)
            print(f'Successfully processed: {ligand_file}')
        except Exception as e:
            print(f'Error processing: {ligand_file}')
            print(f'Error details: {str(e)}')

%vina --config /content/7y89/DOCKING/config_file --receptor /content/7y89/7y89_receptor.pdbqt --ligand /content/7y89/DOCKING/'CHEMBL53914.pdbqt' --out /content/7y89/EXPERIMENTAL/'docking_CHEMBL53914.pdbqt' | tee /content/7y89/DOCKING/'CHEMBL53914.pdbqt'_output.log
%vina --config /content/7y89/DOCKING/config_file --receptor /content/7y89/7y89_receptor.pdbqt --ligand /content/7y89/DOCKING/'CHEMBL3132881.pdbqt' --out /content/7y89/EXPERIMENTAL/'docking_CHEMBL3132881.pdbqt' | tee /content/7y89/DOCKING/'CHEMBL3132881.pdbqt'_output.log
%vina --config /content/7y89/DOCKING/config_file --receptor /content/7y89/7y89_receptor.pdbqt --ligand /content/7y89/DOCKING/'CHEMBL3264006.pdbqt' --out /content/7y89/EXPERIMENTAL/'docking_CHEMBL3264006.pdbqt' | tee /content/7y89/DOCKING/'CHEMBL3264006.pdbqt'_output.log
%vina --config /content/7y89/DOCKING/config_file --receptor /content/7y89/7y89_receptor.pdbqt --ligand /content/7y89/DOCKING/'CHEMBL3921330.pdbqt' --out /content/7y89/EXPERIMENTAL/'docking_CHEM

"        try:\n            os.chdir('/content/vina')\n            subprocess.call(vina, shell=True)\n            print(f'Successfully processed: {ligand_file}')\n        except Exception as e:\n            print(f'Error processing: {ligand_file}')\n            print(f'Error details: {str(e)}')"

In [ ]:
%vina --version

AutoDock Vina v1.2.5


In [ ]:
!obabel /content/ligand/'3-(2-carboxyethyl)-6-chloro-1H-indole-2-carboxylic Acid.sdf' -O /content/'3-(2-carboxyethyl)-6-chloro-1H-indole-2-carboxylic Acid.mol2' --gen3d --best --canonical --minimize --ff GAFF --steps 10000 --sd

In [ ]:
!prepare_ligand4.py -l /content/EPA.mol2 -o /content/7y89/DOCKING/'EPA.pdbqt' -U nphs_lps -v

In [ ]:
view=py3Dmol.view()
view.setBackgroundColor('white')
#Loading the target protein as PDB file
view.addModel(open('/content/7y89/7y89.pdb', 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'spectrum'}})
view.zoomTo()
#Loading the docking pose
pocket_dir = '/content/7y89/7y89_out/pockets'
for pocket_file in os.listdir(pocket_dir):
  if pocket_file.endswith('.pdb'):
    pocket_path = os.path.join(pocket_dir, pocket_file)

    view.addModel(open(pocket_path, 'r').read(),'pdb')
    view.setStyle({'model':1},{'stick':{'colorscheme':'greenCarbon'}})

view.show()